In [ ]:
import os
import binascii
from web3 import Web3, HTTPProvider

from pydex_app.constants import NETWORK_INFO

NETWORK_ID = os.environ.get("NETWORK_ID")
PRIVATE_KEY = binascii.a2b_hex(os.environ.get("PRIVATE_KEY"))
WEB3_RPC_URL = os.environ.get("WEB3_RPC_URL")

web3_provider = HTTPProvider(WEB3_RPC_URL)
w3 = Web3(web3_provider)
acct = w3.eth.account.privateKeyToAccount(PRIVATE_KEY)
MY_ADDRESS = acct.address.lower()
print(
    "NETWORK:",  NETWORK_INFO[NETWORK_ID]["name"],
    "\nRPC_URL", WEB3_RPC_URL,
    "\nADDRESS IN USE:", MY_ADDRESS)

In [ ]:
import json
from zero_ex.order_utils import asset_data_utils as adu

ADDRESS_BOOK = {
    "4": {
      'OptionsRegistry': Web3.toChecksumAddress('0xa5b14070af70f56fc0b3216045e53d3224bb0172'),
    },
  };

option_registry_interface = json.load(open("../artifacts/OptionsRegistry.json"))

option_registry = w3.eth.contract(
    address=ADDRESS_BOOK[NETWORK_ID]["OptionsRegistry"],
    abi=option_registry_interface["abi"],
#     bytecode=option_registry_interface["bytecode"],
)
markets = option_registry.functions.getMarkets().call()
for market_address in markets:
    market_info = option_registry.functions.getMarket(market_address).call()
    print(
        "id: {5} | topic:{0}\n\tstrike: {1}\n\texpiry: {2}\n\tlong_token: '{3}'\n\tshort_token: '{4}'\n".format(
        *market_info))

In [ ]:
# VETH_TOKEN = "0xc4abc01578139e2105d9c9eba0b0aa6f6a60d082"
# VETH_ASSET_DATA = adu.encode_erc20_asset_data(VETH_TOKEN)
# LONG_TOKEN = "0x358b48569a4a4ef6310c1f1d8e50be9d068a50c6"
# LONG_ASSET_DATA = adu.encode_erc20_asset_data(LONG_TOKEN)
# SHORT_TOKEN = "0x4a1f5f67c1cf90f176496aa548382c78921ae9f1"
# SHORT_ASSET_DATA = adu.encode_erc20_asset_data(SHORT_TOKEN)
# print(VETH_TOKEN, VETH_ASSET_DATA, LONG_TOKEN, LONG_ASSET_DATA, SHORT_TOKEN, SHORT_ASSET_DATA)

VETH_TOKEN = "0xc4abc01578139e2105d9c9eba0b0aa6f6a60d082"
VETH_ASSET_DATA = "0xf47261b0000000000000000000000000c4abc01578139e2105d9c9eba0b0aa6f6a60d082" 
LONG_TOKEN = "0x358b48569a4a4ef6310c1f1d8e50be9d068a50c6"
LONG_ASSET_DATA = "0xf47261b0000000000000000000000000358b48569a4a4ef6310c1f1d8e50be9d068a50c6" 
SHORT_TOKEN = "0x4a1f5f67c1cf90f176496aa548382c78921ae9f1"
SHORT_ASSET_DATA = "0xf47261b00000000000000000000000004a1f5f67c1cf90f176496aa548382c78921ae9f1"

full_asset_set_data = {
    "LONG": LONG_ASSET_DATA,
    "SHORT": SHORT_ASSET_DATA,
}

In [ ]:
from pydex_client.client import PyDexClient

client = PyDexClient(
    network_id=NETWORK_ID,
    web3_rpc_url=WEB3_RPC_URL,
    private_key=PRIVATE_KEY,
    pydex_api_url="http://localhost:3000",
)

client.get_orderbook(
    base_asset_data=VETH_ASSET_DATA,
    quote_asset_data=LONG_ASSET_DATA,
    full_set_asset_data=full_asset_set_data
)

In [ ]:
import time

from decimal import Decimal
from zero_ex.json_schemas import assert_valid
from pydex_app.db_models import SignedOrder
from pydex_app.utils import to_base_unit_amount
from pydex_app.config import NULL_ADDRESS


def make_signed_order(
    asset,
    qty,
    price,
    side,
    maker_address=MY_ADDRESS,
    expiration_time_secs=600,
    maker_fee="0",
    taker_fee="0",
    salt=None,
    taker_address=NULL_ADDRESS,
    fee_recipient_address=NULL_ADDRESS,
    sender_address=NULL_ADDRESS,
    exchange_address="0xbce0b5f6eb618c565c3e5f5cd69652bbc279f44e", # Rinkeby Exchange Address
    w3_instance=w3,
    private_key=PRIVATE_KEY,

):
    """ Convenience function for making valid orders to buy or sell
    SHORT or LONG assets against VETH.
    
    Keyword Arguments:
    asset -- str from {'LONG', 'SHORT'} to index into `full_asset_set_data`
    qty -- how much of the ticker asset you want to buy against VETH
    price -- Always in units of LONG or SHORT asset per VETH
    side -- str from {'BUY', 'SELL'}
    maker_address -- your address (defaults to MY_ADDRESS)
    """
    OPTION_ASSET_DATA = full_asset_set_data[asset]
    if side == 'BUY':
        maker_asset_data = VETH_ASSET_DATA
        taker_asset_data = OPTION_ASSET_DATA
        maker_amount = to_base_unit_amount(qty * price)
        taker_amount = to_base_unit_amount(qty)
    elif side == 'SELL':
        maker_asset_data = OPTION_ASSET_DATA
        taker_asset_data = VETH_ASSET_DATA
        maker_amount = to_base_unit_amount(qty)
        taker_amount = to_base_unit_amount(qty * price)
    else:
        raise Exception("side must be one of {'BUY', 'SELL'}")
    
    if not salt:
        # TODO(CM): Is it reasonable to pick the salt like this?
        import random
        salt = "{:.0f}".format(Decimal(random.uniform(0, 9223372036854775807)))
    if not isinstance(maker_fee, str):
        maker_fee = to_base_unit_amount(maker_fee)
    if not isinstance(taker_fee, str):
        taker_fee = to_base_unit_amount(taker_fee)
    expiration_time_secs = int(time.time() + expiration_time_secs)
    
    order = SignedOrder()
    order.maker_address = maker_address
    order.taker_address = taker_address
    order.fee_recipient_address = fee_recipient_address
    order.sender_address = sender_address
    order.maker_asset_amount = maker_amount
    order.taker_asset_amount = taker_amount
    order.maker_fee = "{:.0f}".format(Decimal(maker_fee))
    order.taker_fee = "{:.0f}".format(Decimal(taker_fee))
    order.expiration_time_secs = expiration_time_secs
    order.salt = salt
    order.maker_asset_data = maker_asset_data
    order.taker_asset_data = taker_asset_data
    order.exchange_address = exchange_address
    order.signature = w3_instance.eth.account.signHash(
        order.update().hash, private_key=private_key
    )["signature"].hex()
    assert_valid(order.to_json(), "/signedOrderSchema")
    return order

def sign_and_post_order(order):
    base_url = "http://localhost:3000/v2/"
    print(f"POSTING {order}")
    response = requests.post(
        "{}{}".format(base_url, "order"),
        json=order.to_json()
    )
    return response

In [ ]:
order = make_signed_order(
    asset="LONG",
    qty=0.0001,
    price=0.45,
    side="BUY",
)
order

In [ ]:
from pydex_app.db_models import SignedOrder
from pprint import pprint

orderbook = client.get_orderbook(
    base_asset_data=VETH_ASSET_DATA,
    quote_asset_data=LONG_ASSET_DATA,
    full_set_asset_data=full_asset_set_data
)
# pprint(orderbook)

def print_order_book(orderbook):
    print("BIDS:---------")
    bids = [SignedOrder.from_json(order_json["order"]).update()
            for order_json in orderbook["bids"]["records"]]
    print("\n".join([str(bid) for bid in bids]))
    print("ASKS:---------")
    asks = [SignedOrder.from_json(order_json["order"]).update()
            for order_json in orderbook["asks"]["records"]]
    print("\n".join([str(bid) for bid in asks]))
    return bids, asks
    
bids, asks = print_order_book(orderbook)

In [ ]:
make_signed_order(
        asset="LONG",
        qty=0.0001,
        price=0.45,
        side="BUY",
    ).to_json()

In [ ]:
for side in ["BUY", "SELL"]:
    order = make_signed_order(
        asset="LONG",
        qty=0.0001,
        price=0.45,
        side=side,
    )
    print(json.dumps(order.to_json(include_hash=True)))
    res = sign_and_post_order(order)
    print(res)

In [ ]:
res.json()

In [ ]:
# from pydex_app.db_models import SignedOrder
# list(SignedOrder.query.filter_by(
#     maker_asset_data=VETH_ASSET_DATA))

In [ ]:
from pydex_app.order_watcher_client import OrderWatcherClient

owc = OrderWatcherClient(
    on_msg=print,
    on_error=print,
)
owc.run()

In [ ]:
for order in bids + asks:
    res = owc._rpc(method="ADD_ORDER", params={"signedOrder": order.to_json()})
    print(">>>>", res)

In [ ]:
res = owc._rpc(method="GET_STATS")
res

In [ ]:
for order in bids + asks:
    res = owc._rpc(method="REMOVE_ORDER", params={"orderHash": order.hash})
    print(">>>>", res)

In [ ]:
res = owc._rpc(method="GET_STATS")
res